In [1]:
import torch
import torchaudio
from torchaudio.pipelines import MMS_FA as bundle
from typing import List
import IPython
import matplotlib.pyplot as plt
from pypinyin import pinyin, lazy_pinyin, Style
from pydub import AudioSegment

print(torch.__version__)
print(torchaudio.__version__)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

import whisper
from pydub import AudioSegment
import librosa
from opencc import OpenCC


2.4.0+cu118
2.4.0+cu118
cuda


In [2]:
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)

raw_data = "raw_audio\\record.WAV"

model = whisper.load_model("large-v2")
# 使用Whisper進行語音識別
result = model.transcribe(raw_data)
result['segments']

d:\anaconda3\envs\Py39\lib\site-packages\whisper\__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


[{'id': 0,
  'seek': 0,
  'start': 0.0,
  'end': 2.0,
  'text': '你好',
  'tokens': [50364, 26410, 50464],
  'temperature': 0.0,
  'avg_logprob': -0.5360870361328125,
  'compression_ratio': 0.42857142857142855,
  'no_speech_prob': 0.040411192923784256}]

In [3]:
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |   6025 MiB |   8961 MiB |  65967 MiB |  59941 MiB |\n|       from large pool |   6020 MiB |   8952 MiB |  64336 MiB |  58316 MiB |\n|       from small pool |      5 MiB |      8 MiB |   1630 MiB |   1624 MiB |\n|---------------------------------------------------------------------------|\n| Active memory         |   6025 MiB |   8961 MiB |  65967 MiB |  59941 MiB |\n|       from large pool |   6020 MiB |   8952 MiB |

In [4]:
sentence =[]
cc = OpenCC('s2t')

for i in range(len(result['segments'])):
    sentence.append([cc.convert(result['segments'][i]['text'].lower().replace('》','').replace('《','').replace('%','').replace('。','').replace('?','').replace('【','').replace('】','').replace('-','').replace('.','').replace(',', '').replace('6','六').replace('4','四').replace('2','二').replace('9','九').replace('8','八').replace('5','五').replace('3','三').replace('0','零').replace('1','一').replace('7','七').replace(' ','').replace('、','')), 
                     result['segments'][i]['start'], 
                     result['segments'][i]['end']])
   

sentence

[['你好', 0.0, 2.0]]

In [5]:
audio = AudioSegment.from_file(raw_data)
for i in range(len(sentence)):
    audio_clip = audio[sentence[i][1] *1000: sentence[i][2]*1000]
    audio_clip.export(f"sentences\\{(sentence[i][0].lower())}.wav", format="wav")

In [6]:
model = bundle.get_model()
model.to(device)

tokenizer = bundle.get_tokenizer()
aligner = bundle.get_aligner()

def compute_alignments(waveform: torch.Tensor, transcript: List[str]):
    with torch.inference_mode():
        emission, _ = model(waveform.to(device))
        token_spans = aligner(emission[0], tokenizer(transcript))
    return emission, token_spans

def _score(spans):
    return sum(s.score * len(s) for s in spans) / sum(len(s) for s in spans)


def plot_alignments(waveform, token_spans, emission, transcript, sample_rate=bundle.sample_rate):
    ratio = waveform.size(1) / emission.size(1) / sample_rate

    fig, axes = plt.subplots(2, 1)
    axes[0].imshow(emission[0].detach().cpu().T, aspect="auto")
    axes[0].set_title("Emission")
    axes[0].set_xticks([])

    axes[1].specgram(waveform[0], Fs=sample_rate)
    for t_spans, chars in zip(token_spans, transcript):
        t0, t1 = t_spans[0].start, t_spans[-1].end
        axes[0].axvspan(t0 - 0.5, t1 - 0.5, facecolor="None", hatch="/", edgecolor="white")
        axes[1].axvspan(ratio * t0, ratio * t1, facecolor="None", hatch="/", edgecolor="white")
        axes[1].annotate(f"{_score(t_spans):.2f}", (ratio * t0, sample_rate * 0.51), annotation_clip=False)

        for span, char in zip(t_spans, chars):
            t0 = span.start * ratio
            axes[1].annotate(char, (t0, sample_rate * 0.55), annotation_clip=False)

    axes[1].set_xlabel("time [second]")
    fig.tight_layout()

def preview_word(waveform, spans, num_frames, transcript, sample_rate):
    ratio = waveform.size(1) / num_frames
    x0 = int(ratio * spans[0].start)
    x1 = int(ratio * spans[-1].end)
    #print(f"{transcript} ({_score(spans):.2f}): {x0 / sample_rate:.3f} - {x1 / sample_rate:.3f} sec")
    time_StarAndEnd = [ x0 / sample_rate, x1/sample_rate] # 回傳單個字的起始時間與結束時間
    segment = waveform[:, x0:x1]
    #return IPython.display.Audio(segment.numpy(), rate=sample_rate)
    return time_StarAndEnd
    

In [7]:
for i in range(0,len(sentence)):#len(sentence)-1
    text_normalized = ' '.join(lazy_pinyin(sentence[i][0]))#將文字轉為沒有音調的拼音，lazy_pinyin是陣列所以要再join成字串

    waveform, sample_rate = librosa.load(f"sentences\\{sentence[i][0]}.wav")
    waveform_tensor = torch.tensor(waveform).unsqueeze(0)

    transcript = text_normalized.split()
    emission, token_spans = compute_alignments(waveform_tensor, transcript)
    num_frames = emission.size(1)


    #plot_alignments(waveform, token_spans, emission, transcript)

    print("Raw Transcript: ", sentence[i][0])
    print("Normalized Transcript: ", text_normalized)
    IPython.display.Audio(waveform, rate=sample_rate)

    text_raw = sentence[i][0]
    word_start_end = []
    pinyin_tone = pinyin(text_raw, style=Style.TONE3, heteronym=False)
    for j in range(len(transcript)):#len(transcript)
        timeStartEnd = preview_word(waveform_tensor, token_spans[j], num_frames, transcript[j], sample_rate)
        word_start_end.append([pinyin_tone[j][0], timeStartEnd[0], timeStartEnd[1]])
    print(word_start_end)

    audio = AudioSegment.from_file(f"sentences\\{sentence[i][0]}.wav")
    file_name = sentence[i][0]
    for k in range(len(word_start_end)):
        segment_audio = audio[word_start_end[k][1] *1000: word_start_end[k][2]*1000]
        segment_audio.export(f"data\\{file_name}-{k}_{word_start_end[k][0]}.wav", format="wav")
    print('------------------------------------------')

d:\anaconda3\envs\Py39\lib\site-packages\torchaudio\models\wav2vec2\components.py:305: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Raw Transcript:  你好
Normalized Transcript:  ni hao
[['ni3', 0.5131065759637188, 0.6010884353741497], ['hao3', 0.8649886621315193, 1.1435374149659865]]
------------------------------------------
